In [ ]:
import pandas as pd

import pandas as pd

# ---- 1. Load counts table (Excel) ----
counts = pd.read_excel(
    "/Users/hornung_comp1/Downloads/MCE_Bioactive_Compounds_HEK293T_10μM_Counts.xlsx",
    engine="openpyxl", header=1
)

# ---- 2. Load meta table (CSV) ----
meta = pd.read_csv(
    "/Users/hornung_comp1/meta_MCE_with_smiles_pubchem_parallel.csv",
)

counts.head(), meta.head()

# Merge so each count row knows its treatment & metadata
df = counts.merge(
    meta,
    left_on='Sample_id',
    right_on='unique_ID',
    how='left'
)


In [ ]:
import pandas as pd
import numpy as np

# ------------------------------------------------
# 0. Define gene columns
# ------------------------------------------------
non_gene_cols = [
    'Sample_id',
    'unique_ID',
    'experiment_no',
    'sample_plate',
    'sample_row',
    'sample_column',
    'cell_id',
    'pert_itime',
    'pert_idose',
    'sample',
    'treatment',
    'Catalog Number',
    'Compound name',
    'CAS Number',
    'smiles'
]

gene_cols = [c for c in df.columns if c not in non_gene_cols]

# ------------------------------------------------
# 1. How many raw DMSO rows do we have?
# ------------------------------------------------
n_dmso_raw = (df['treatment'] == "DMSO").sum()
print(f"Number of raw DMSO rows (before averaging): {n_dmso_raw}")

# ------------------------------------------------
# 2. Average raw counts per treatment (including DMSO)
# ------------------------------------------------
avg_counts = (
    df
    .groupby('treatment')[gene_cols]
    .mean()
    .reset_index()
)

# Check how many DMSO rows after averaging (should be 1)
n_dmso_avg = (avg_counts['treatment'] == "DMSO").sum()
print(f"Number of averaged DMSO rows (after groupby): {n_dmso_avg}")

# ------------------------------------------------
# 3. Normalize all treatments (incl. DMSO): CPM → log1p
# ------------------------------------------------
cpm_all = (
    avg_counts[gene_cols]
    .div(avg_counts[gene_cols].sum(axis=1), axis=0)
    * 1e6
)

log1p_cpm_all = np.log1p(cpm_all)

# ------------------------------------------------
# 4. Extract DMSO baseline (per gene, from the ONE averaged DMSO row)
# ------------------------------------------------
dmso_mask = avg_counts['treatment'] == "DMSO"

if n_dmso_avg == 0:
    raise ValueError("No DMSO row found in avg_counts! Check 'treatment' labels.")
elif n_dmso_avg > 1:
    raise ValueError("More than one DMSO row found in avg_counts. Something is off.")

dmso_vec = log1p_cpm_all.loc[dmso_mask].iloc[0]  # 1D vector, length = #genes

# ------------------------------------------------
# 5. Subtract DMSO from ALL treatments
# ------------------------------------------------
log1p_cpm_delta_all = log1p_cpm_all.subtract(dmso_vec, axis=1)

# Attach treatment back
avg_features = pd.concat(
    [avg_counts[['treatment']], log1p_cpm_delta_all],
    axis=1
)

# ------------------------------------------------
# 6. Build per-treatment meta for compounds with SMILES
#    (DMSO will drop out here because smiles is NaN)
# ------------------------------------------------
smiles_per_treat = (
    df[['treatment', 'smiles', 'Catalog Number', 'Compound name', 'CAS Number']]
    .dropna(subset=['smiles'])               # keep only rows where SMILES is present
    .drop_duplicates(subset=['treatment'])   # one meta row per treatment
)

# Inner merge: only treatments that have a SMILES somewhere
avg_with_smiles = avg_features.merge(
    smiles_per_treat,
    on='treatment',
    how='inner'
)

print(f"Rows after merging with SMILES (DMSO dropped here): {avg_with_smiles.shape[0]}")

# ------------------------------------------------
# 7. Add dose + seq_platform
# ------------------------------------------------
avg_with_smiles['pert_idose'] = 10          # 10 µM
avg_with_smiles['seq_platform'] = 0         # arbitrary platform code

# ------------------------------------------------
# 8. Reorder columns: meta first, then genes
# ------------------------------------------------
meta_first = [
    'treatment',
    'smiles',
    'Catalog Number',
    'Compound name',
    'CAS Number',
    'pert_idose',
    'seq_platform'
]

gene_cols_final = [c for c in avg_with_smiles.columns if c not in meta_first]

avg_with_smiles = avg_with_smiles[meta_first + gene_cols_final]

# ------------------------------------------------
# 9. Final ML table: keep smiles, dose, platform + delta(DMSO) genes
# ------------------------------------------------
meta_for_ml = ['smiles', 'pert_idose', 'seq_platform']

final_df = pd.concat(
    [avg_with_smiles[meta_for_ml], avg_with_smiles[gene_cols_final]],
    axis=1
)

# ------------------------------------------------
# 10. Save to new file
# ------------------------------------------------
out_path = "/Users/hornung_comp1/MCE_HEK293T_10uM_log1pCPM_deltaDMSO.csv"
final_df.to_csv(out_path, index=False)

print(f"Saved: {out_path}")
print(f"Shape: {final_df.shape}")
print("Columns (first 10):", final_df.columns[:10].tolist())
